In [1]:
import pandas as pd
import numpy as np
import os
import pyspark
import pyspark.sql.functions as F

In [2]:
if sc:
    sc.stop()

NameError: name 'sc' is not defined

In [3]:
sc = pyspark.SparkContext().getOrCreate()
spark = pyspark.sql.SparkSession(sc)

Exception: Java gateway process exited before sending its port number

In [5]:
path_raw_imdb = "data/raw_data/imdb.dataset/"
path_parsed_imdb = "data/parsed_data/imdb.db"

## name.basics
### name.basic
#### PySpark

In [69]:
name_basics = spark.read.csv(
    os.path.join(path_raw_imdb, "name.basics.tsv"), 
    sep="\t", 
    header=True, 
    nullValue="\\N", 
    inferSchema=True
)

In [70]:
name_basics.write.parquet(os.path.join(path_parsed_imdb, "name_basics"), mode="overwrite")

In [71]:
name_basics = spark.read.parquet(os.path.join(path_parsed_imdb, "name_basics"))
name_basics.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- deathYear: integer (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)



In [72]:
name_basics.show(5)

+---------+----------------+---------+---------+--------------------+--------------------+
|   nconst|     primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+----------------+---------+---------+--------------------+--------------------+
|nm1380630|Giorgos Kyriakos|     null|     null|producer,producti...|tt5392738,tt03571...|
|nm1380631|           Kysse|     null|     null|             actress|           tt0289752|
|nm1380632| Kämy Kämäräinen|     null|     null|               actor| tt1802456,tt0168123|
|nm1380633| Calle Kärnekull|     null|     null|              stunts|           tt0098796|
|nm1380634|    Imre Kõszegi|     1944|     null|    music_department|           tt0152796|
+---------+----------------+---------+---------+--------------------+--------------------+
only showing top 5 rows



#### pandas

In [ ]:
name_basics = pd.read_csv(
    os.path.join(path_raw, "name.basics.tsv"), 
    sep="\t", 
    na_values="\\N", 
    index_col=0, 
)

In [ ]:
name_basics.info()

In [ ]:
name_basics.head()

In [ ]:
name_basics.to_parquet(os.path.join(path_parsed_imdb, "name_basics.parquet"), engine="pyarrow")

### link.name.basics.titles
#### PySpark

In [73]:
name_basics = spark.read.parquet(os.path.join(path_parsed_imdb, "name_basics"))

In [74]:
name_basics.show(5)

+---------+----------------+---------+---------+--------------------+--------------------+
|   nconst|     primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+----------------+---------+---------+--------------------+--------------------+
|nm1380630|Giorgos Kyriakos|     null|     null|producer,producti...|tt5392738,tt03571...|
|nm1380631|           Kysse|     null|     null|             actress|           tt0289752|
|nm1380632| Kämy Kämäräinen|     null|     null|               actor| tt1802456,tt0168123|
|nm1380633| Calle Kärnekull|     null|     null|              stunts|           tt0098796|
|nm1380634|    Imre Kõszegi|     1944|     null|    music_department|           tt0152796|
+---------+----------------+---------+---------+--------------------+--------------------+
only showing top 5 rows



In [75]:
link_name_basics_titles = (
    name_basics
    .select("nconst", F.explode(F.split(name_basics.knownForTitles, ",")).alias("tconst"))
)

In [76]:
link_name_basics_titles.show(5)

+---------+---------+
|   nconst|   tconst|
+---------+---------+
|nm1380630|tt5392738|
|nm1380630|tt0357123|
|nm1380630|tt1111843|
|nm1380630|tt6304968|
|nm1380631|tt0289752|
+---------+---------+
only showing top 5 rows



In [77]:
link_name_basics_titles.write.parquet(os.path.join(path_parsed_imdb, "link_name_basics_titles"), mode="overwrite")

## title.basics
##### PySpark

In [40]:
title_basics = spark.read.csv(
    os.path.join(path_raw_imdb, "title.basics.tsv"), 
    sep="\t", 
    header=True, 
    nullValue="\\N", 
    inferSchema=True
)

In [41]:
title_basics.write.parquet( os.path.join(path_parsed_imdb, "title_basics"), mode="overwrite")

In [42]:
title_basics = spark.read.parquet( os.path.join(path_parsed_imdb, "title_basics"))
title_basics.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- endYear: integer (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [43]:
title_basics.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   null|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   null|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|   null|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   null|          null|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   null|             1|        Comedy

#### pandas

In [ ]:
title_basics = pd.read_csv(
    os.path.join(path_raw, "title.basics.tsv"), 
    sep="\t", 
    na_values="\\N", 
    index_col=0, 
    dtype={"runtimeMinutes": "object"}
)

In [ ]:
title_basics.head()

In [ ]:
title_basics.info()

In [ ]:
title_basics.runtimeMinutes = title_basics.runtimeMinutes.astype(str)

In [ ]:
title_basics.to_parquet(os.path.join(path_parsed_imdb, "title_basics.parquet"), engine="pyarrow")

## title.ratings

#### PySpark

In [32]:
title_ratings = spark.read.csv(
    os.path.join(path_raw_imdb, "title.ratings.tsv"), 
    sep="\t", 
    header=True, 
    nullValue="\\N", 
    inferSchema=True
)

In [34]:
title_ratings.write.parquet(os.path.join(path_parsed_imdb, "title_ratings"), mode="overwrite")

In [35]:
title_ratings = spark.read.parquet(os.path.join(path_parsed_imdb, "title_ratings"))
title_ratings.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)



In [36]:
title_ratings.show(5)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.6|    1531|
|tt0000002|          6.1|     185|
|tt0000003|          6.5|    1179|
|tt0000004|          6.2|     113|
|tt0000005|          6.1|    1896|
+---------+-------------+--------+
only showing top 5 rows



#### pandas

In [ ]:
title_ratings = pd.read_csv(
    os.path.join(path_raw, "title.ratings.tsv"), 
    sep="\t", 
    na_values="\\N", 
    index_col=0
)

In [ ]:
title_ratings.head()

In [ ]:
title_ratings.info()

In [ ]:
title_ratings.to_parquet(os.path.join(path_parsed_imdb, "title_ratings.parquet"), engine="pyarrow")

## title.principals
#### PySpark

In [27]:
title_principals = spark.read.csv(
    os.path.join(path_raw_imdb, "title.principals.tsv"), 
    sep="\t", 
    header=True, 
    nullValue="\\N", 
    inferSchema=True
)

In [28]:
title_principals.write.parquet(os.path.join(path_parsed_imdb, "title_principals"), mode="overwrite")

In [29]:
title_principals = spark.read.parquet(os.path.join(path_parsed_imdb, "title_principals"))
title_principals.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- nconst: string (nullable = true)
 |-- category: string (nullable = true)
 |-- job: string (nullable = true)
 |-- characters: string (nullable = true)



In [30]:
title_principals.show(5)

+---------+--------+---------+---------------+--------------------+-----------+
|   tconst|ordering|   nconst|       category|                 job| characters|
+---------+--------+---------+---------------+--------------------+-----------+
|tt0000001|       1|nm1588970|           self|                null|["Herself"]|
|tt0000001|       2|nm0005690|       director|                null|       null|
|tt0000001|       3|nm0374658|cinematographer|director of photo...|       null|
|tt0000002|       1|nm0721526|       director|                null|       null|
|tt0000002|       2|nm1335271|       composer|                null|       null|
+---------+--------+---------+---------------+--------------------+-----------+
only showing top 5 rows



#### pandas

In [4]:
path_file = os.path.join(path_raw_imdb, "title.principals.tsv")
chunk_size = 2 ** 16
d_type = {
    "tconst": "object",
    "ordering": "int64",
    "nconst": "object",
    "category": "object",
    "job": "object",
    "characters": "object"
}

title_principals = pd.DataFrame()
_iter = 0
for chunk in pd.read_csv(path_file, sep="\t", na_values="\\N", index_col=0, dtype=d_type, chunksize=chunk_size):
    title_principals = pd.concat([title_principals, chunk])
    _iter += 1
print(_iter)
print(title_principals.shape)
print(title_principals.index)

539
(35314063, 5)
Index(['tt0000001', 'tt0000001', 'tt0000001', 'tt0000002', 'tt0000002',
       'tt0000003', 'tt0000003', 'tt0000003', 'tt0000003', 'tt0000004',
       ...
       'tt9916880', 'tt9916880', 'tt9916880', 'tt9916880', 'tt9916880',
       'tt9916880', 'tt9916880', 'tt9916880', 'tt9916880', 'tt9916880'],
      dtype='object', name='tconst', length=35314063)


In [5]:
title_principals.head()

,ordering,nconst,category,job,characters
tconst,,,,,
tt0000001,1,nm1588970,self,NaN,"[""Herself""]"
tt0000001,2,nm0005690,director,NaN,NaN
tt0000001,3,nm0374658,cinematographer,director of photography,NaN
tt0000002,1,nm0721526,director,NaN,NaN
tt0000002,2,nm1335271,composer,NaN,NaN


In [6]:
title_principals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35314063 entries, tt0000001 to tt9916880
Data columns (total 5 columns):
ordering      int64
nconst        object
category      object
job           object
characters    object
dtypes: int64(1), object(4)
memory usage: 1.6+ GB


In [7]:
title_principals.to_parquet(os.path.join(path_parsed_imdb, "title_principals.parquet"), engine="pyarrow")
#title_principals.write.parquet(os.path.join(path_parsed_imdb, "title_principals.parquet"), mode="overwrite")

## title.crew
### title.crew
#### PySpark

In [54]:
title_crew = spark.read.csv(
    os.path.join(path_raw_imdb, "title.crew.tsv"), 
    sep="\t", 
    header=True, 
    nullValue="\\N", 
    inferSchema=True
)

In [55]:
title_crew.write.parquet(os.path.join(path_parsed_imdb, "title_crew"), mode="overwrite")

In [56]:
title_crew = spark.read.parquet(os.path.join(path_parsed_imdb, "title_crew"))
title_crew.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- directors: string (nullable = true)
 |-- writers: string (nullable = true)



In [57]:
title_crew.show(5)

+---------+-------------------+--------------------+
|   tconst|          directors|             writers|
+---------+-------------------+--------------------+
|tt3093900|               null|                null|
|tt3093902|               null|                null|
|tt3093904|               null|                null|
|tt3093906|nm3983984,nm3985542|nm5844710,nm39839...|
|tt3093908|          nm1787707|           nm1787707|
+---------+-------------------+--------------------+
only showing top 5 rows



#### pandas

In [ ]:
title_crew = pd.read_csv(
    os.path.join(path_raw, "title.crew.tsv"), 
    sep="\t", 
    na_values="\\N", 
    index_col=0
)

In [ ]:
title_crew.head()

In [ ]:
title_crew.info()

In [53]:
title_crew.to_parquet(os.path.join(path_parsed_imdb, "title_crew"), engine="pyarrow")

AttributeError: 'DataFrame' object has no attribute 'to_parquet'

### link.title.directors
#### PySpark

In [58]:
title_crew = spark.read.parquet(os.path.join(path_parsed_imdb, "title_crew"))

In [59]:
title_crew.show(5)

+---------+-------------------+--------------------+
|   tconst|          directors|             writers|
+---------+-------------------+--------------------+
|tt3093900|               null|                null|
|tt3093902|               null|                null|
|tt3093904|               null|                null|
|tt3093906|nm3983984,nm3985542|nm5844710,nm39839...|
|tt3093908|          nm1787707|           nm1787707|
+---------+-------------------+--------------------+
only showing top 5 rows



In [60]:
link_title_directors = (
    title_crew.select("tconst", F.explode(F.split(title_crew.directors, ",")).alias("nconsts"))
)

In [61]:
link_title_directors.show(5)

+---------+---------+
|   tconst|  nconsts|
+---------+---------+
|tt3093906|nm3983984|
|tt3093906|nm3985542|
|tt3093908|nm1787707|
|tt3093916|nm0943944|
|tt3093918|nm0943944|
+---------+---------+
only showing top 5 rows



In [62]:
link_title_directors.write.parquet(os.path.join(path_parsed_imdb, "link_title_directors"), mode="overwrite")

## title.akas
### title.akas
#### PySpark

In [63]:
title_akas = spark.read.csv(
    os.path.join(path_raw_imdb, "title.akas.tsv"), 
    sep="\t", 
    header=True, 
    nullValue="\\N", 
    inferSchema=True
)

In [64]:
title_akas.write.parquet(os.path.join(path_parsed_imdb, "title_akas"), mode="overwrite")

In [65]:
title_akas = spark.read.parquet(os.path.join(path_parsed_imdb, "title_akas"))
title_akas.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- types: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- isOriginalTitle: integer (nullable = true)



In [66]:
title_akas.show(5)

+---------+--------+----------+------+--------+-----------+----------+---------------+
|  titleId|ordering|     title|region|language|      types|attributes|isOriginalTitle|
+---------+--------+----------+------+--------+-----------+----------+---------------+
|tt1172049|       5|Разрушение|    BG|      bg|       null|      null|              0|
|tt1172049|       6| Demolição|    PT|    null|imdbDisplay|      null|              0|
|tt1172049|       7|   Rušenje|    HR|    null|       null|      null|              0|
|tt1172049|       8|Darabokban|    HU|    null|imdbDisplay|      null|              0|
|tt1172049|       9|Destrukcja|    PL|    null|imdbDisplay|      null|              0|
+---------+--------+----------+------+--------+-----------+----------+---------------+
only showing top 5 rows



#### pandas

In [ ]:
title_akas = pd.read_csv(
    os.path.join(path_raw, "title.akas.tsv"), 
    sep="\t", 
    na_values="\\N", 
    index_col=0
)

In [ ]:
title_akas.head()

In [ ]:
title_akas.info()

In [ ]:
title_akas.to_parquet(os.path.join(path_parsed_imdb, "title_akas.parquet"), engine="pyarrow")